In [ ]:
# This notebook plays with the MNIST digits dataset, for which I use the
# parser https://pypi.org/project/python-mnist/
# and the official data source http://www.itl.nist.gov/iaui/vip/cs_links/EMNIST/gzip.zip
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


from mnist import MNIST
mndata = MNIST('~/git/python-mnist/data')
images, labels = mndata.load_training()
testX, testY = mndata.load_testing()

In [ ]:
digits

In [ ]:
digits = np.array(images)

In [ ]:
digits.shape

In [ ]:
p = np.reshape(digits, newshape=( 60000, 28,28))

In [ ]:
p.shape

In [ ]:
plt.imshow(p[4,::])

In [ ]:
plt.subplot(221)
plt.imshow(p[0,::])
plt.subplot(222)
plt.imshow(p[1,::])
plt.subplot(223)
plt.imshow(p[2,::])
plt.subplot(224)
plt.imshow(p[3,::])
plt.savefig("MNIST.png", dpi=300, bbox_inches="tight")

In [ ]:
ones = np.where(np.array(labels)==8)

In [ ]:
ones[0]

In [ ]:
images[ones[0]]

In [ ]:
labels[0:100]

In [ ]:
sample = digits[0:1000,:]

In [ ]:
import seaborn as sb
sb.heatmap(sample)
plt.xlabel("pixels")
plt.ylabel("samples")

In [ ]:
# Wait, this isn't quite what I want.  I want to reorder columns (pixel positions) and rows (sample digits).
sb.clustermap(sample)


In [ ]:
# A little busy, maybe only look at 100 samples?
sb.clustermap(sample[0:100,:])

In [ ]:
sb.clustermap(sample[0:100,:], row_colors=labels[0:100])

In [ ]:
# sb.clustermap wants a list of color names for row_colors.  Fine.  

h = {0: "black", 1:"brown", 2:"red", 3:"orange", 4:"yellow",
    5:"green", 6:"blue", 7:"purple", 8:"grey", 9:"white"}
colorlabels100 = []
for l in labels[0:100]:
    colorlabels100.append(h[l])
colorlabels100

In [ ]:
clustermap = sb.clustermap(sample[0:100,:], row_colors=colorlabels100)

In [ ]:
# The clustermap library function gives me the data in attributes of the return value:
dir(clustermap)

In [ ]:
# There is data, which is what I gave it, 
plt.imshow(clustermap.data)

In [ ]:
# And data2d, which has been reordered
plt.imshow(clustermap.data2d, aspect="auto")

In [ ]:
# I can plot what all the digits look like on top of each other
plt.imshow(np.array(clustermap.data.sum(axis=0)).reshape( (28,28)))

In [ ]:
# And it turns out the indexes for the row- and column- reordering are buried in ax_col_dendrogram:
dir(clustermap.ax_col_dendrogram)

In [ ]:
# To get the reordering of the rows and columns,
# https://github.com/mwaskom/seaborn/issues/1207
# I need clustermap.dendrogram_row.reordered_ind
clustermap.dendrogram_row.reordered_ind [ 0:20]

In [ ]:
clustermap.dendrogram_col.reordered_ind[0:10]

In [ ]:
digits.shape

In [ ]:
# So let's get some pieces of this clustergram and plot them on a 28x28 pixel grid:
middlepixels = clustermap.dendrogram_col.reordered_ind[300:600]
endpixels = clustermap.dendrogram_col.reordered_ind[723:]

In [ ]:
middle_image = np.zeros((28,28))
end_image = np.zeros((28,28))

In [ ]:
middle_image.reshape((784,))[middlepixels] = 1
end_image.reshape((784,))[endpixels] = 1

In [ ]:
# These are the pixels that were clustered
plt.subplot(121)
plt.imshow(middle_image)
plt.subplot(122)
plt.imshow(end_image)

In [ ]:
# Asking clustermap to compare my rows and columns...
# Let us try PCA, linear-algebra-only dimension reduction approach.

from sklearn.decomposition import PCA
pca=PCA(n_components=30)
pca.fit(sample.T)

In [ ]:
pca.components_.shape

In [ ]:
plt.scatter(pca.components_[0,:], pca.components_[1,:])

In [ ]:
plt.scatter(pca.components_[0,:], pca.components_[1,:], color=colorlabels100)

In [ ]:
# This is a sample of 1000 points. 
h = {0: "black", 1:"brown", 2:"red", 3:"orange", 4:"yellow",
    5:"green", 6:"blue", 7:"purple", 8:"grey", 9:"white"}
colorlabels1k = []
for l in labels[0:1000]:
    colorlabels1k.append(h[l])
colorlabels1k[0:50]

colorlabelsall = []
for l in labels:
    colorlabelsall.append(h[l])
colorlabelsall[0:50]

In [ ]:
plt.scatter(pca.components_[0,:], pca.components_[1,:], color=colorlabels1k)
plt.xlabel("PC 1    "+"{:.1f}".format(pca.explained_variance_ratio_[0] * 100) + "%"); 
plt.ylabel("PC 2    "+"{:.1f}".format(pca.explained_variance_ratio_[1] * 100) + "%"); 

In [ ]:
plt.scatter(pca.components_[2,:], pca.components_[3,:], color=colorlabels1k)
plt.xlabel("PC 3    "+"{:.1f}".format(pca.explained_variance_ratio_[2] * 100) + "%"); 
plt.ylabel("PC 4    "+"{:.1f}".format(pca.explained_variance_ratio_[3] * 100) + "%"); 

In [ ]:
# A little disappointing, 1000 digits fed into PCA and plotted. 
# Let us try TSNE on this 1000 sample by 30-dimensional PCA

In [ ]:
from sklearn.manifold import TSNE
X_embedded = TSNE(n_components=2, learning_rate=.1, init='random').fit(pca.components_.T)

In [ ]:
X_embedded

In [ ]:
dir(X_embedded)

In [ ]:
X_embedded.embedding_.shape

In [ ]:
# We want to plot embedding_[:,1] against embedding_[:,0] 
plt.scatter(X_embedded.embedding_[:,0], X_embedded.embedding_[:,1])

In [ ]:
# And add label colors
plt.scatter(X_embedded.embedding_[:,0], X_embedded.embedding_[:,1], color=colorlabels1k)

In [ ]:
# This is a little more satisfying than [blind] PCA, but 
# less than perfect.  

In [ ]:
# Fit 50-component PCA on all the data
pca50=PCA(n_components=50)
pca50.fit(digits.T)

In [ ]:
plt.scatter(pca50.components_[0,:], pca50.components_[1,:], color=colorlabelsall)
plt.xlabel("PC 1    "+"{:.1f}".format(pca50.explained_variance_ratio_[0] * 100) + "%"); 
plt.ylabel("PC 2    "+"{:.1f}".format(pca50.explained_variance_ratio_[1] * 100) + "%"); 

In [ ]:
pca50.components_.shape

In [ ]:
# Fit TSNE on subset of the big PCA output, but since TSNE takes too long for 10^{4} points, use only 
# 1000 samples
TSNE1K = TSNE(n_components=2, learning_rate=.1, init='random').fit(pca50.components_[:,0:1000].T)

In [ ]:
plt.scatter(TSNE1K.embedding_[:,0], TSNE1K.embedding_[:,1], color=colorlabelsall[0:1000])

In [ ]:
TSNE4K = TSNE(n_components=2, learning_rate=.1, init='random').fit(pca50.components_[:,0:4000].T)

In [ ]:
plt.scatter(TSNE4K.embedding_[:,0], TSNE4K.embedding_[:,1], color=colorlabelsall[0:4000])

In [ ]:
# Wait, this is qualitatively different from TSNE with n=1k 

# We can see that the TSNE procedure for drawing pretty 2d pictures that 
# reflect concentrations of point density in high dimensional spaces
# is sensitive to number of points and tuning parameters; 
# and be warned, it is not deterministic, so you don't get the
# same clouds each time you run it.  But it sure is pretty.

In [ ]:
TSNE10K = TSNE(n_components=2, learning_rate=1.5, init="random").fit(pca50.components_[:,0:10000].T)

In [ ]:
plt.scatter(TSNE10K.embedding_[:,0], TSNE10K.embedding_[:,1], color=colorlabelsall[0:10000])

In [ ]:
digits

In [ ]:
# The image of all the pixels flattened out is jumbled -- it is a mixture of the digits.
plt.imshow(digits, aspect="auto")

In [ ]:
# I can sort by the labels, and show them grouped by digit.
digitsordered = digits[np.argsort(labels),:]
digits.shape

In [ ]:
plt.imshow(digitsordered, aspect="auto")